![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/healthcare-nlp/08.6.Text_Classification_with_FewShotClassifier.ipynb)

## Colab Setup

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install -q johnsnowlabs

In [ ]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

In [ ]:
from johnsnowlabs import nlp, medical

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
nlp.settings.enforce_versions=True
nlp.install(refresh_install=True)

In [ ]:
from johnsnowlabs import nlp, medical
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Automatically load license data and start a session with all jars user has access to
spark = nlp.start()


In [ ]:
spark

In [ ]:
# Installing tensorflow-addons
! pip install -q tensorflow-addons

In [ ]:
from pyspark.sql import DataFrame
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pyspark.sql as SQL
from pyspark import keyword_only
from sklearn.metrics import classification_report

# Few Shot Classification

The `FewShotClassifierApproach` and `FewShotClassifierModel` annotators are new additions to the set of annotators available in the NLP library. These annotators specifically target few-shot classification tasks, which involve training a model to make accurate predictions with limited labeled data.

These new annotators provide a valuable capability for handling scenarios where labeled data is scarce or expensive to obtain. By effectively utilizing limited labeled examples, the few-shot classification approach enables the creation of models that can generalize and classify new instances accurately, even with minimal training data.

The FewShotClassifier is designed to process sentence embeddings as input. It generates category annotations, providing labels along with confidence scores that range from 0 to 1. Input annotation types supported by this model include `SENTENCE_EMBEDDINGS`, while the output annotation type is `CATEGORY`.

In [ ]:
data = [
    ["ADE_positive", 'Both PAN and methotrexate have been independently demonstrated to cause sensorineural hearing loss.'],
    ["ADE_positive", 'Increased lash length, thickness, and pigmentation are well-documented side effects of prostaglandin analog glaucoma drops.'],
    ["ADE_positive", 'We reviewed the records of 3 patients with lymphoproliferative disorders who experienced acute coronary syndromes associated with their initial infusion of rituximab.'],
    ["ADE_positive", 'A 58-year-old woman with rheumatoid arthritis (RA) developed fever, skin eruptions, leukocytopenia, and thrombocytopenia, 3 weeks after treatment with sulfasalazine.'],
    ["ADE_positive", 'Adrenal suppression in a fetus due to administration of methylprednisolone has hitherto been rarely published.'],
    ["ADE_negative", 'Serum concentration of cerivastatin at 6 h after taking the last dose (0.15 mg) was 8062.5 ng/L, which was almost 5.7 times higher than that of normal persons.'],
    ["ADE_negative", 'The usual treatment includes quick relief bronchodilator medications of the sympathomimetic class and controller medications that may include the long-acting inhaled bronchodilator salmeterol.'],
    ["ADE_negative", 'Pathogenic mechanisms for the development of pseudomembranous colitis and the epidemiology of this condition in patients with AIDS are discussed.'],
    ["ADE_negative", 'On the basis of the clinico-radiologic presentation, a pulmonary hemorrhage was likely to occur; so to clarify the origin of this process, a complete serologic examination was performed but all the antibodies were negative.'],
    ["ADE_negative", 'I report a patient who developed the syndrome during treatment for schizophrenia with the antipsychotic agent molindone hydrochloride.']
]

In [ ]:
train_data = spark.createDataFrame(data).toDF("label","text")
train_data.show(truncate=100)

+------------+----------------------------------------------------------------------------------------------------+
|       label|                                                                                                text|
+------------+----------------------------------------------------------------------------------------------------+
|ADE_positive| Both PAN and methotrexate have been independently demonstrated to cause sensorineural hearing loss.|
|ADE_positive|Increased lash length, thickness, and pigmentation are well-documented side effects of prostaglan...|
|ADE_positive|We reviewed the records of 3 patients with lymphoproliferative disorders who experienced acute co...|
|ADE_positive|A 58-year-old woman with rheumatoid arthritis (RA) developed fever, skin eruptions, leukocytopeni...|
|ADE_positive|Adrenal suppression in a fetus due to administration of methylprednisolone has hitherto been rare...|
|ADE_negative|Serum concentration of cerivastatin at 6 h after taking th

In [ ]:
document_asm = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("sentence")

sentence_embeddings = nlp.BertSentenceEmbeddings\
    .pretrained("sbiobert_base_cased_mli","en","clinical/models")\
    .setInputCols(["sentence"])\
    .setOutputCol("sentence_embeddings")

graph_builder = medical.TFGraphBuilder()\
    .setModelName("fewshot_classifier")\
    .setInputCols(["sentence_embeddings"]) \
    .setLabelColumn("label")\
    .setGraphFolder("/tmp")\
    .setGraphFile("log_reg_graph.pb")\

few_shot_approach = medical.FewShotClassifierApproach()\
    .setLabelColumn("label")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("prediction")\
    .setModelFile(f"/tmp/log_reg_graph.pb")\
    .setEpochsNumber(10)\
    .setBatchSize(1)\
    .setLearningRate(0.001)

pipeline = nlp.Pipeline(
    stages=[
        document_asm,
        sentence_embeddings,
        graph_builder,
        few_shot_approach
])

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]


In [ ]:
%%time
model = pipeline.fit(train_data)

TF Graph Builder configuration:
Model name: fewshot_classifier
Graph folder: /tmp
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 768, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid'}
fewshot_classifier graph exported to /tmp/log_reg_graph.pb
CPU times: user 3.27 s, sys: 367 ms, total: 3.64 s
Wall time: 9.91 s


In [ ]:
model.stages[-1].write().overwrite().save("/tmp/few_shot_model")

In [ ]:
few_shot_model = medical.FewShotClassifierModel.load("/tmp/few_shot_model")

## with LightPipeline

In [ ]:
lpipeline = nlp.LightPipeline(nlp.Pipeline(
                stages=[
                    document_asm,
                    sentence_embeddings,
                    few_shot_model.setMultiClass(False)
                ]).fit(train_data))

In [ ]:
tests = [
    'Bleomycin pneumonitis potentiated by oxygen administration.',
    'Enzymes derived from two different bacterial sources (Escherichia coli and Erwinia carotovora) are in common use.',
    'These cases and others quoted indicate that dependence on pressurized aerosol bronchodilators can and does occur in young people.',
    'PVT during chemotherapy in children is a rare event and appears to be closely related to intensive chemotherapy containing busulfan and to be associated with HVOD.',
]

In [ ]:
for i, r in enumerate(lpipeline.fullAnnotate(tests)):
    print(i, r)


0 {'sentence': [Annotation(document, 0, 58, Bleomycin pneumonitis potentiated by oxygen administration., {}, [])], 'sentence_embeddings': [Annotation(sentence_embeddings, 0, 58, Bleomycin pneumonitis potentiated by oxygen administration., {'sentence': '0', 'token': 'Bleomycin pneumonitis potentiated by oxygen administration.', 'pieceId': '-1', 'isWordStart': 'true'}, [])], 'prediction': [Annotation(category, 0, 0, ADE_positive, {'confidence': '0.72445786'}, [])]}
1 {'sentence': [Annotation(document, 0, 112, Enzymes derived from two different bacterial sources (Escherichia coli and Erwinia carotovora) are in common use., {}, [])], 'sentence_embeddings': [Annotation(sentence_embeddings, 0, 112, Enzymes derived from two different bacterial sources (Escherichia coli and Erwinia carotovora) are in common use., {'sentence': '0', 'token': 'Enzymes derived from two different bacterial sources (Escherichia coli and Erwinia carotovora) are in common use.', 'pieceId': '-1', 'isWordStart': 'true'}

In [ ]:
for i, r in enumerate(lpipeline.fullAnnotate(tests)):
    print(i ,r["prediction"][0].result)

0 ADE_positive
1 ADE_negative
2 ADE_negative
3 ADE_positive


In [ ]:
for i, r in enumerate(lpipeline.fullAnnotate(tests)):
    print(i ,r["prediction"][0],r["sentence"][0].result )

0 Annotation(category, 0, 0, ADE_positive, {'confidence': '0.72445786'}, []) Bleomycin pneumonitis potentiated by oxygen administration.
1 Annotation(category, 0, 0, ADE_negative, {'confidence': '0.6759418'}, []) Enzymes derived from two different bacterial sources (Escherichia coli and Erwinia carotovora) are in common use.
2 Annotation(category, 0, 0, ADE_negative, {'confidence': '0.5071792'}, []) These cases and others quoted indicate that dependence on pressurized aerosol bronchodilators can and does occur in young people.
3 Annotation(category, 0, 0, ADE_positive, {'confidence': '0.52034104'}, []) PVT during chemotherapy in children is a rare event and appears to be closely related to intensive chemotherapy containing busulfan and to be associated with HVOD.


In [ ]:
lpipeline.fullAnnotate('After 1 week of nefazodone therapy the patient experienced headache, confusion, and "gray areas" in her vision, without abnormal ophthalmologic findings.')

[{'sentence': [Annotation(document, 0, 152, After 1 week of nefazodone therapy the patient experienced headache, confusion, and "gray areas" in her vision, without abnormal ophthalmologic findings., {}, [])],
  'sentence_embeddings': [Annotation(sentence_embeddings, 0, 152, After 1 week of nefazodone therapy the patient experienced headache, confusion, and "gray areas" in her vision, without abnormal ophthalmologic findings., {'sentence': '0', 'token': 'After 1 week of nefazodone therapy the patient experienced headache, confusion, and "gray areas" in her vision, without abnormal ophthalmologic findings.', 'pieceId': '-1', 'isWordStart': 'true'}, [])],
  'prediction': [Annotation(category, 0, 0, ADE_positive, {'confidence': '0.72127557'}, [])]}]

In [ ]:
lpipeline_multi = nlp.LightPipeline(nlp.Pipeline(
                      stages=[
                          document_asm,
                          sentence_embeddings,
                          few_shot_model.setMultiClass(True)
                      ]).fit(train_data))

In [ ]:
for i, r in enumerate(lpipeline_multi.fullAnnotate(tests)):
    print(i,r)


0 {'sentence': [Annotation(document, 0, 58, Bleomycin pneumonitis potentiated by oxygen administration., {}, [])], 'sentence_embeddings': [Annotation(sentence_embeddings, 0, 58, Bleomycin pneumonitis potentiated by oxygen administration., {'sentence': '0', 'token': 'Bleomycin pneumonitis potentiated by oxygen administration.', 'pieceId': '-1', 'isWordStart': 'true'}, [])], 'prediction': [Annotation(category, 0, 0, ADE_positive, {'confidence': '0.72445786'}, []), Annotation(category, 0, 0, ADE_negative, {'confidence': '0.23561996'}, [])]}
1 {'sentence': [Annotation(document, 0, 112, Enzymes derived from two different bacterial sources (Escherichia coli and Erwinia carotovora) are in common use., {}, [])], 'sentence_embeddings': [Annotation(sentence_embeddings, 0, 112, Enzymes derived from two different bacterial sources (Escherichia coli and Erwinia carotovora) are in common use., {'sentence': '0', 'token': 'Enzymes derived from two different bacterial sources (Escherichia coli and Erwi

In [ ]:
for i, r in enumerate(lpipeline_multi.fullAnnotate(tests)):
    print(i, r["prediction"])


0 [Annotation(category, 0, 0, ADE_positive, {'confidence': '0.72445786'}, []), Annotation(category, 0, 0, ADE_negative, {'confidence': '0.23561996'}, [])]
1 [Annotation(category, 0, 0, ADE_positive, {'confidence': '0.5473867'}, []), Annotation(category, 0, 0, ADE_negative, {'confidence': '0.6759418'}, [])]
2 [Annotation(category, 0, 0, ADE_positive, {'confidence': '0.45842287'}, []), Annotation(category, 0, 0, ADE_negative, {'confidence': '0.5071792'}, [])]
3 [Annotation(category, 0, 0, ADE_positive, {'confidence': '0.52034104'}, []), Annotation(category, 0, 0, ADE_negative, {'confidence': '0.38408756'}, [])]


In [ ]:
lpipeline_multi.fullAnnotate('After 1 week of nefazodone therapy the patient experienced headache, confusion, and "gray areas" in her vision, without abnormal ophthalmologic findings.')

[{'sentence': [Annotation(document, 0, 152, After 1 week of nefazodone therapy the patient experienced headache, confusion, and "gray areas" in her vision, without abnormal ophthalmologic findings., {}, [])],
  'sentence_embeddings': [Annotation(sentence_embeddings, 0, 152, After 1 week of nefazodone therapy the patient experienced headache, confusion, and "gray areas" in her vision, without abnormal ophthalmologic findings., {'sentence': '0', 'token': 'After 1 week of nefazodone therapy the patient experienced headache, confusion, and "gray areas" in her vision, without abnormal ophthalmologic findings.', 'pieceId': '-1', 'isWordStart': 'true'}, [])],
  'prediction': [Annotation(category, 0, 0, ADE_positive, {'confidence': '0.72127557'}, []),
   Annotation(category, 0, 0, ADE_negative, {'confidence': '0.360577'}, [])]}]

## with transform

In [ ]:
test_data = spark.createDataFrame([
    ['After 1 week of nefazodone therapy the patient experienced headache, confusion, and "gray areas" in her vision, without abnormal ophthalmologic findings.'],
    ['Possible linkage of amprenavir with intracranial bleeding in an HIV-infected hemophiliac.'],
    ['Results Radiographic responses to sirolimus were observed in all patients.'],
    ['Obtaining appropriate cultures can be critical in making the diagnosis and directing treatment.']
]).toDF("text")

In [ ]:
result = model.transform(test_data)

result.select("text","prediction").show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------+
|text                                                                                                                                                     |prediction                                                      |
+---------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------+
|After 1 week of nefazodone therapy the patient experienced headache, confusion, and "gray areas" in her vision, without abnormal ophthalmologic findings.|[{category, 0, 0, ADE_positive, {confidence -> 0.72127557}, []}]|
|Possible linkage of amprenavir with intracranial bleeding in an HIV-infected hemophiliac.                          

In [ ]:
model_multi = nlp.Pipeline(
                  stages=[
                      document_asm,
                      sentence_embeddings,
                      few_shot_model.setMultiClass(True)
                  ]).fit(train_data)

result_2 = model_multi.transform(test_data)

result_2.select("text","prediction").show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                                     |prediction                                                                                                                      |
+---------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------+
|After 1 week of nefazodone therapy the patient experienced headache, confusion, and "gray areas" in her vision, without abnormal ophthalmologic 

## with word embeddings

In [ ]:
tokenizer = nlp.Tokenizer() \
    .setInputCols(["sentence"]) \
    .setOutputCol("token")

word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical","en","clinical/models")\
    .setInputCols(["sentence","token"])\
    .setOutputCol("word_embeddings")

sentence_embeddings = nlp.SentenceEmbeddings() \
    .setInputCols(["sentence", "word_embeddings"]) \
    .setOutputCol("sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")

pipeline = nlp.Pipeline(
    stages=[
        document_asm,
        tokenizer,
        word_embeddings,
        sentence_embeddings,
        few_shot_approach
    ])

model = pipeline.fit(spark.createDataFrame([["",""]]).toDF("label","text"))

lpipeline = nlp.LightPipeline(model)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]


In [ ]:
lpipeline.fullAnnotate('Nephrotic syndrome associated with interferon-beta-1b therapy for multiple sclerosis.')

[{'word_embeddings': [Annotation(word_embeddings, 0, 8, Nephrotic, {'isOOV': 'false', 'pieceId': '-1', 'isWordStart': 'true', 'token': 'Nephrotic', 'sentence': '0'}, []),
   Annotation(word_embeddings, 10, 17, syndrome, {'isOOV': 'false', 'pieceId': '-1', 'isWordStart': 'true', 'token': 'syndrome', 'sentence': '0'}, []),
   Annotation(word_embeddings, 19, 28, associated, {'isOOV': 'false', 'pieceId': '-1', 'isWordStart': 'true', 'token': 'associated', 'sentence': '0'}, []),
   Annotation(word_embeddings, 30, 33, with, {'isOOV': 'false', 'pieceId': '-1', 'isWordStart': 'true', 'token': 'with', 'sentence': '0'}, []),
   Annotation(word_embeddings, 35, 52, interferon-beta-1b, {'isOOV': 'false', 'pieceId': '-1', 'isWordStart': 'true', 'token': 'interferon-beta-1b', 'sentence': '0'}, []),
   Annotation(word_embeddings, 54, 60, therapy, {'isOOV': 'false', 'pieceId': '-1', 'isWordStart': 'true', 'token': 'therapy', 'sentence': '0'}, []),
   Annotation(word_embeddings, 62, 64, for, {'isOOV': '

# MTSamples Dataset

In [ ]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/mtsamples_classifier.csv

In [ ]:
spark_df = spark.read.csv("mtsamples_classifier.csv", header = True)

spark_df.show(10,truncate=100)

+----------------+----------------------------------------------------------------------------------------------------+
|        category|                                                                                                text|
+----------------+----------------------------------------------------------------------------------------------------+
|Gastroenterology| PROCEDURES PERFORMED: Colonoscopy. INDICATIONS: Renewed symptoms likely consistent with active f...|
|Gastroenterology| OPERATION 1. Ivor-Lewis esophagogastrectomy. 2. Feeding jejunostomy. 3. Placement of two right-s...|
|Gastroenterology| PREOPERATIVE DIAGNOSES: 1. Gastroesophageal reflux disease. 2. Chronic dyspepsia. POSTOPERATIVE ...|
|Gastroenterology| PROCEDURE: Colonoscopy. PREOPERATIVE DIAGNOSES: Rectal bleeding and perirectal abscess. POSTOPER...|
|Gastroenterology| PREOPERATIVE DIAGNOSIS: Right colon tumor. POSTOPERATIVE DIAGNOSES: 1. Right colon cancer. 2. As...|
|Gastroenterology| PREOPERATIVE DIAGNOSI

In [ ]:
spark_df.groupBy("category").count().show()

+----------------+-----+
|        category|count|
+----------------+-----+
|         Urology|  115|
|       Neurology|  143|
|      Orthopedic|  223|
|Gastroenterology|  157|
+----------------+-----+



In [ ]:
(trainingData, testData) = spark_df.randomSplit([0.8, 0.2], seed = 42)
(trainingData_part1, trainingData_part2) = trainingData.randomSplit([0.5, 0.5], seed = 42)

print("trainingData_with_embeddings:       ",trainingData.count())
print("testData_with_embeddings:           ",testData.count())
print("trainingData_part1_with_embeddings: ",trainingData_part1.count())

trainingData_with_embeddings:        536
testData_with_embeddings:            102
trainingData_part1_with_embeddings:  280


We will extract [sbiobert_base_cased_mli](https://nlp.johnsnowlabs.com/2020/11/27/sbiobert_base_cased_mli_en.html) embeddings which has 768 dimension output and use this embeddings in the model training.

In [ ]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

bert_sent = nlp.BertSentenceEmbeddings.pretrained("sbiobert_base_cased_mli", 'en','clinical/models')\
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

embeddings_pipeline = nlp.Pipeline(
    stages = [document_assembler,
              bert_sent])

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]


In [ ]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)\
                                                  .select("text","category","sentence_embeddings")

trainingData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+----------------+------------------------------------------------------------+
|                                                        text|        category|                                         sentence_embeddings|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
| ADMISSION DIAGNOSIS: Symptomatic cholelithiasis. DISCHAR...|Gastroenterology|[{sentence_embeddings, 0, 2228,  ADMISSION DIAGNOSIS: Sym...|
| ADMITTING DIAGNOSES: Hiatal hernia, gastroesophageal ref...|Gastroenterology|[{sentence_embeddings, 0, 3237,  ADMITTING DIAGNOSES: Hia...|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
only showing top 2 rows



In [ ]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)\
                                                  .select("text","category","sentence_embeddings")

testData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+----------------+------------------------------------------------------------+
|                                                        text|        category|                                         sentence_embeddings|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
| ADMITTING DIAGNOSIS: Gastrointestinal bleed. HISTORY OF ...|Gastroenterology|[{sentence_embeddings, 0, 3978,  ADMITTING DIAGNOSIS: Gas...|
| CHIEF COMPLAINT: Dysphagia and hematemesis while vomitin...|Gastroenterology|[{sentence_embeddings, 0, 6515,  CHIEF COMPLAINT: Dysphag...|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
only showing top 2 rows



In [ ]:
trainingData_part1_with_embeddings = embeddings_pipeline.fit(trainingData_part1).transform(trainingData_part1)\
                                                  .select("text","category","sentence_embeddings")

trainingData_part2_with_embeddings = embeddings_pipeline.fit(trainingData_part2).transform(trainingData_part2)\
                                                  .select("text","category","sentence_embeddings")

### ClassifierDL

In [ ]:
log_folder="classifier_dl_logs_bert"

In [ ]:
classifier_dl = nlp.ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("prediction_class")\
        .setLabelColumn("category")\
        .setBatchSize(8)\
        .setMaxEpochs(10)\
        .setLr(0.002)\
        .setDropout(0.1)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)

classifier_dl_pipeline = nlp.Pipeline(stages=[classifier_dl])

In [ ]:
%%time
clfDL_model_bert = classifier_dl_pipeline.fit(trainingData_with_embeddings)

CPU times: user 427 ms, sys: 56.6 ms, total: 484 ms
Wall time: 1min 22s


In [ ]:
preds = clfDL_model_bert.transform(testData_with_embeddings)

preds_df = preds.select("category","text","prediction_class.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print(classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

Gastroenterology       1.00      0.84      0.91        25
       Neurology       0.82      0.82      0.82        22
      Orthopedic       0.79      0.86      0.82        35
         Urology       0.81      0.85      0.83        20

        accuracy                           0.84       102
       macro avg       0.85      0.84      0.85       102
    weighted avg       0.85      0.84      0.84       102



In [ ]:
results_df = pd.DataFrame(columns=["macro-f1-score","weighted-f1-score","accuracy"])

res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["ClassifierDL_full_Data"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

In [ ]:
results_df

,macro-f1-score,weighted-f1-score,accuracy
ClassifierDL_full_Data,0.845603,0.844888,0.843137


### ClassifierDL partial

In [ ]:
log_folder="classifier_dl_logs_bert"

In [ ]:
classifier_dl = nlp.ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("prediction_class")\
        .setLabelColumn("category")\
        .setBatchSize(8)\
        .setMaxEpochs(10)\
        .setLr(0.002)\
        .setDropout(0.1)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)

classifier_dl_pipeline = nlp.Pipeline(stages=[classifier_dl])

In [ ]:
%%time

clfDL_model_bert = classifier_dl_pipeline.fit(trainingData_part1_with_embeddings)

CPU times: user 235 ms, sys: 39.6 ms, total: 274 ms
Wall time: 46.5 s


In [ ]:
preds = clfDL_model_bert.transform(testData_with_embeddings)

preds_df = preds.select("category","text","prediction_class.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

Gastroenterology       0.76      0.88      0.81        25
       Neurology       0.78      0.82      0.80        22
      Orthopedic       0.91      0.86      0.88        35
         Urology       0.88      0.75      0.81        20

        accuracy                           0.83       102
       macro avg       0.83      0.83      0.83       102
    weighted avg       0.84      0.83      0.83       102



In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["ClassifierDL_partial_Data"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

In [ ]:
results_df

,macro-f1-score,weighted-f1-score,accuracy
ClassifierDL_partial_Data,0.826995,0.834009,0.833333


## Few Shot partial

In [ ]:
graph_builder = medical.TFGraphBuilder()\
    .setModelName("fewshot_classifier")\
    .setInputCols(["sentence_embeddings"]) \
    .setLabelColumn("category")\
    .setGraphFolder("/tmp")\
    .setGraphFile("log_reg_graph.pb")\

few_shot_approach = medical.FewShotClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("prediction")\
    .setModelFile(f"/tmp/log_reg_graph.pb")\
    .setEpochsNumber(10)\
    .setBatchSize(8)\
    .setLearningRate(0.002)

pipeline = nlp.Pipeline(
    stages=[
        graph_builder,
        few_shot_approach
    ])

In [ ]:
%%time
model = pipeline.fit(trainingData_part1_with_embeddings)

TF Graph Builder configuration:
Model name: fewshot_classifier
Graph folder: /tmp
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 768, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid'}
fewshot_classifier graph exported to /tmp/log_reg_graph.pb
CPU times: user 1.13 s, sys: 103 ms, total: 1.23 s
Wall time: 1min 56s


In [ ]:
preds = model.transform(testData_with_embeddings)

In [ ]:
from sklearn.metrics import  classification_report

preds_df = preds.select("category","text","prediction.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

Gastroenterology       0.88      0.88      0.88        25
       Neurology       0.78      0.82      0.80        22
      Orthopedic       0.81      0.86      0.83        35
         Urology       0.88      0.75      0.81        20

        accuracy                           0.83       102
       macro avg       0.84      0.83      0.83       102
    weighted avg       0.84      0.83      0.83       102



In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["FewShot_partial_Data"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

In [ ]:
results_df

,macro-f1-score,weighted-f1-score,accuracy
ClassifierDL_full_Data,0.881651,0.883697,0.882353
ClassifierDL_partial_Data,0.127737,0.175326,0.343137
FewShot_partial_Data,0.831036,0.833166,0.833333


# Few Shot Clasification Model

|index|model|
|-----:|:-----|
|1|[few_shot_classifier_age_group_sbiobert_cased_mli](https://nlp.johnsnowlabs.com/2023/08/17/few_shot_classifier_age_group_sbiobert_cased_mli_en.html)
|2|[few_shot_classifier_patient_complaint_sbiobert_cased_mli](https://nlp.johnsnowlabs.com/2023/08/30/few_shot_classifier_patient_complaint_sbiobert_cased_mli_en.html)

1.	`few_shot_classifier_age_group_sbiobert_cased_mli`

In [ ]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

bert_sent = nlp.BertSentenceEmbeddings.pretrained("sbiobert_base_cased_mli", "en", "clinical/models")\
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

few_shot_classifier = medical.FewShotClassifierModel.pretrained("few_shot_classifier_age_group_sbiobert_cased_mli", "en", "clinical/models")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("prediction")

clf_Pipeline = nlp.Pipeline(stages=[
    document_assembler,
    bert_sent,
    few_shot_classifier
])

data = spark.createDataFrame([
    ["""A patient presented with complaints of chest pain and shortness of breath. The medical history revealed the patient had a smoking habit for over 30 years, and was diagnosed with hypertension two years ago. After a detailed physical examination, the doctor found a noticeable wheeze on lung auscultation and prescribed a spirometry test, which showed irreversible airway obstruction. The patient was diagnosed with Chronic obstructive pulmonary disease (COPD) caused by smoking."""],
 ["""Hi, wondering if anyone has had a similar situation. My 1 year old daughter has the following; loose stools/ pale stools, elevated liver enzymes, low iron.  5 months and still no answers from drs. """],
 ["""Hi have chronic gastritis from 4 month(confirmed by endoscopy).I do not have acid reflux.Only dull ache above abdomen and left side of chest.I am on reberprozole and librax.My question is whether chronic gastritis is curable or is it a lifetime condition?I am loosing hope because this dull ache is not going away.Please please reply"""]
    ]).toDF("text")

result = clf_Pipeline.fit(data).transform(data)

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
few_shot_classifier_age_group_sbiobert_cased_mli download started this may take some time.
[OK!]


In [ ]:
few_shot_classifier.getClasses()

['Unknown', 'Adult', 'Child']

In [ ]:
result.show()

+--------------------+--------------------+--------------------+--------------------+
|                text|            document| sentence_embeddings|          prediction|
+--------------------+--------------------+--------------------+--------------------+
|A patient present...|[{document, 0, 47...|[{sentence_embedd...|[{category, 0, 0,...|
|Hi, wondering if ...|[{document, 0, 19...|[{sentence_embedd...|[{category, 0, 0,...|
|Hi have chronic g...|[{document, 0, 33...|[{sentence_embedd...|[{category, 0, 0,...|
+--------------------+--------------------+--------------------+--------------------+



In [ ]:
result.select('prediction.result','text').show(truncate=150)

+---------+------------------------------------------------------------------------------------------------------------------------------------------------------+
|   result|                                                                                                                                                  text|
+---------+------------------------------------------------------------------------------------------------------------------------------------------------------+
|  [Adult]|A patient presented with complaints of chest pain and shortness of breath. The medical history revealed the patient had a smoking habit for over 30...|
|  [Child]|Hi, wondering if anyone has had a similar situation. My 1 year old daughter has the following; loose stools/ pale stools, elevated liver enzymes, l...|
|[Unknown]|Hi have chronic gastritis from 4 month(confirmed by endoscopy).I do not have acid reflux.Only dull ache above abdomen and left side of chest.I am o...|
+---------+-----------

2.	`few_shot_classifier_patient_complaint_sbiobert_cased_mli`

In [ ]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

bert_sent = nlp.BertSentenceEmbeddings.pretrained("sbiobert_base_cased_mli", "en", "clinical/models")\
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

few_shot_classifier = medical.FewShotClassifierModel.pretrained("few_shot_classifier_patient_complaint_sbiobert_cased_mli", "en", "clinical/models")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("prediction")

clf_Pipeline = nlp.Pipeline(stages=[
    document_assembler,
    bert_sent,
    few_shot_classifier
])

data = spark.createDataFrame([["""The Medical Center is a large state of the art hospital facility with great doctors, nurses, technicians and receptionists.  Service is top notch, knowledgeable and friendly.  This hospital site has plenty of parking"""],
 ["""My gf dad wasn’t feeling well so we decided to take him to this place cus it’s his insurance and we waited for a while and mind that my girl dad couldn’t breath good while the staff seem not to care and when they got to us they said they we’re gonna a take some blood samples and they made us wait again and to see the staff workers talking to each other and laughing taking there time and not seeming to care about there patience, while we were in the lobby there was another guy who told us they also made him wait while he can hardly breath and they left him there to wait my girl dad is coughing and not doing better and when the lady came in my girl dad didn’t have his shirt because he was hot and the lady came in said put on his shirt on and then left still waiting to get help rn"""]]).toDF("text")

result = clf_Pipeline.fit(data).transform(data)

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
few_shot_classifier_patient_complaint_sbiobert_cased_mli download started this may take some time.
[OK!]


In [ ]:
few_shot_classifier.getClasses()

['No_Complaint', 'Complaint']

In [ ]:
result.show()


+--------------------+--------------------+--------------------+--------------------+
|                text|            document| sentence_embeddings|          prediction|
+--------------------+--------------------+--------------------+--------------------+
|The Medical Cente...|[{document, 0, 21...|[{sentence_embedd...|[{category, 0, 0,...|
|My gf dad wasn’t ...|[{document, 0, 78...|[{sentence_embedd...|[{category, 0, 0,...|
+--------------------+--------------------+--------------------+--------------------+



In [ ]:
result.select('prediction.result', 'text').show(truncate=150)


+--------------+------------------------------------------------------------------------------------------------------------------------------------------------------+
|        result|                                                                                                                                                  text|
+--------------+------------------------------------------------------------------------------------------------------------------------------------------------------+
|[No_Complaint]|The Medical Center is a large state of the art hospital facility with great doctors, nurses, technicians and receptionists.  Service is top notch, ...|
|   [Complaint]|My gf dad wasn’t feeling well so we decided to take him to this place cus it’s his insurance and we waited for a while and mind that my girl dad co...|
+--------------+------------------------------------------------------------------------------------------------------------------------------------------------